<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective" data-toc-modified-id="Objective-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-for-the-collections" data-toc-modified-id="Data-for-the-collections-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data for the collections</a></span></li><li><span><a href="#Connecting-and-creating-the-database-and-collections" data-toc-modified-id="Connecting-and-creating-the-database-and-collections-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Connecting and creating the database and collections</a></span></li></ul></div>

## Objective

The objective of this notebook is to create the database and the collections from which the project API (QuoteAPI) will draw

## Libraries

In [50]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
import sys

## Data for the collections

In [38]:
literature = pd.read_csv("Literature.csv", sep=';')
biology = pd.read_csv("Biology.csv", sep=';')
philosophy = pd.read_csv("Philosophy.csv", sep=';')

In [39]:
#df to dict
biol = biology.to_dict(orient = "records")
phil = philosophy.to_dict(orient = "records")
lit = literature.to_dict(orient = "records")

##  Connecting and creating the database and collections

In [40]:
client = MongoClient()
db = client.QuoteAPI

In [41]:
biology =  db.Biology
philosophy =  db.Philosophy
literature =  db.Literature

In [42]:
db.Biology.insert_many(biol)
db.Philosophy.insert_many(phil)
db.Literature.insert_many(lit)    

In [94]:
#Para combinar todas las collecciones en una sola, donde solo tengamos la informacion de las frases y los autores
y = db.Biology.aggregate( [
   { '$unionWith': "Philosophy"},
   { '$unionWith': "Literature"},   
   { '$project': { '_id': 1, "Quote":1, "Author":1, "Category":1, "Gender":1}},
   { '$out' : "Quote_Author" } ])

In the cells below I try to combine all the citations for an author into a single id. It didn't work out for me at all, so I didn't add it to the API.

In [95]:
y = db.Biology.aggregate( [
   { '$unionWith': "Philosophy"},
   { '$unionWith': "Literature"},   
   { "$group": {
    "_id": "$Author",
    "Quotes": {
      "$push": {
        "Quote": "$$ROOT"  #nos permite mantener todos los documentos dl
      }
    }
  }}, 
   { '$out' : "All_Quotes" } ])

In [87]:
list(db.All_Quotes.find({'_id': ' Socrates'}))

[{'_id': ' Socrates',
  'Quotes': [{'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fbd'),
     'Quote': 'The unexamined life is not worth living ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}},
   {'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fd0'),
     'Quote': 'There is only one good, knowledge, and one evil, ignorance ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}},
   {'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fef'),
     'Quote': 'The only thing I know is that I know nothing ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}}]}]